In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import cv2, os, time
from moviepy.editor import *
from tqdm import tqdm
from pathlib import Path

def video2image( video_path:str,
                 output_dir:str  = "./output/",
                 output_subDir_name:str = None,
                 init_frame:int  = 0,
                 final_frame:int = None,
                 skip:int        = 1,
                 output_ext:str  = "jpg",
                 frame_proc_func = lambda x:x,
                ):
    '''
        動画ファイルからフレームを切り出して画像（jpg）で保存する

        args:
            video_path(str)        : 対象の動画データの場所
            output_dir(str)        : 切り出したフレームの保存先(中に動画ファイル名と同じディレクトリが生成される)
            output_subDir_name(str): 画像を保存するサブディレクトリの名前
            init_frame(int)        : 切り取り対象とする初期フレーム
            final_frame(int)       : 切り取り対象とする最終フレーム
            skip(int)              : skipフレームごとにフレームを保存
            output_ext(str)        : フレームを保存する際の画像の拡張子（"."なし）
            frame_proc_func(func)  : 切り出しフレームに対して前処理を行うための関数

        return:
            None
    '''

    video_path = Path(video_path)
    if output_subDir_name is None:
        output_subDir_name = video_path.stem

    output_dir = Path(output_dir)/Path(output_subDir_name)
    os.makedirs(output_dir, exist_ok=True)

    base_path = output_dir/video_path.stem

    if not video_path.exists():
        raise Exception("this file does not exist.")

    print("{} --> {}_**.{}".format(video_path, base_path, output_ext))

    video_clip = VideoFileClip(str(video_path))

    total_frame = int(video_clip.reader.nframes)
    fps         = video_clip.fps
    digit       = len(str(total_frame))

    if (final_frame is None) or (final_frame > total_frame):
        final_frame = total_frame
    if final_frame < 0:
        final_frame = total_frame + (final_frame)

    print("total_frame: {}, fps: {}, digit: {}".format(total_frame, fps, digit))
    print("init_frame: {}, final_frame: {}".format(init_frame, final_frame))

    for i, frame in tqdm(enumerate(video_clip.iter_frames()), initial=-1, total=final_frame, desc="video2image"):
        time.sleep(0.1)
        if i < init_frame:
            continue
        if i > final_frame:
            break

        frame = frame_proc_func(frame)
        cv2.imwrite('{}_{}.{}'.format(base_path, str(i).zfill(digit), output_ext), frame[:,:,::-1])

        del frame

    return None


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import shutil
video_path = "../../sample_data/sample_mp4.mp4"
output_dir = "./output/"

video2image(video_path=video_path,
            output_dir=output_dir, 
            output_subDir_name="test_name",
            init_frame=50, 
            final_frame=100,
            output_ext="jpg",
            )


..\..\sample_data\sample_mp4.mp4 --> output\test_name\sample_mp4_**.jpg
total_frame: 325, fps: 25.0, digit: 3
init_frame: 50, final_frame: 100


video2image: 100%|██████████| 100/100 [00:11<00:00,  8.71it/s]
